# Progetto CV: A.A 2020-2021

## Candidato: Scuderi Ivan, 216635

## Notebook di Test

### Import preliminari

In [1]:
SEED = 9246

import os
import math
import json
import time
from datetime import datetime
import collections
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import models

from PIL import Image

from skimage import io
import pandas as pd
import numpy as np
import cv2
np.random.seed(SEED)

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support


import torchsummary
from libsvm import svmutil
from brisque import BRISQUE

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


CUDA = torch.cuda.is_available()
device = torch.device("cuda" if CUDA else "cpu")

if CUDA:
    print('Run on cuda!')

Run on cuda!


### Funzioni di utilità

In [2]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def test(model, topk=10):
    print('Start testing the model ( topk = ',topk,')')
    predictions = []
    y_true = []
    model.eval()
    with torch.no_grad():
        for x, y in testDataLoader:
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            _, idx = y_pred.topk(topk, dim=1)
            y_pred = torch.zeros_like(y_pred)
            
            y_pred.scatter_(1, idx, 1)
            predictions.append(y_pred.cpu())

            y_true.append(y.cpu())
    y_true, predictions = torch.cat(y_true, axis=0), torch.cat(predictions, axis=0)
    pr,re,f1,_ =precision_recall_fscore_support(y_true, predictions, average='micro')
    print("Test END: -- m_precision = ",np.round(pr,2), " -- m_recall = ",np.round(re,2)," -- m_f1 = ",np.round(f1,2))

### Caricamento Dati

In [3]:
DATASET_DIR = r'D:\1997i\Desktop\Magistrale\esami\secondo anno\secondo semestre\Analisi di Immagini e Video\progetto\progetto_2021_dataset_preproc\progetto_2021_dataset_labeled'
DRIVE_FOLDER = r'D:\1997i\Documenti\-- Notebook Analisi Immagini e Video\progetto\final\model_checkpoint'

In [4]:
JSON_DATA = os.path.join(DATASET_DIR, 'train_test_split_dict.json')

with open(JSON_DATA) as fp:
    dataset_json = json.load(fp)

labels = set()

for k in dataset_json.values():
    for lable_list in k.values():
        for v in lable_list:
            labels.add(v)
            
label_idx = {v: i for i, v in enumerate(sorted(labels))}

print('Numero di etichette del dataset labeled: ', len(label_idx))

Numero di etichette del dataset labeled:  85


In [5]:
from utils_proc_shuffle import VideoDataset

limit = 20

datasetTest = VideoDataset(DATASET_DIR, dataset_json['test'], label_idx, limit, train=False, shuffle=False)

print('Dimensione Dataset Test : ', len(datasetTest))

Dimensione Dataset Test :  1112


In [6]:
batch_size = 16

testDataLoader = torch.utils.data.DataLoader(datasetTest, 
                                             batch_size=batch_size, 
                                             shuffle=True, 
                                             num_workers=2)

print('Numero di batch complessivi: ', len(testDataLoader))

Numero di batch complessivi:  70


### Definizione e Caricamento Modello

In [7]:
CHECKPOINT = os.path.join(DRIVE_FOLDER, 'model.checkpoint')
MODELFILE = os.path.join(DRIVE_FOLDER, 'model.pth')

def load_model(model):
    if os.path.exists(MODELFILE):
        model.load_state_dict(torch.load(MODELFILE))
        print("Caricamento modello avvenuto con successo!")

In [8]:
net = models.vgg19(pretrained = True)

class Identity(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, x):
        return x

net.fc2 = Identity()

fully_connected = nn.Sequential(
    nn.Conv1d(20,20,kernel_size=(3),padding=1,stride=2),
    nn.ReLU(),
    nn.BatchNorm1d(20),
    nn.Conv1d(20,32,kernel_size=(3),padding=1,stride=2),
    nn.ReLU(),
    nn.BatchNorm1d(32),
    nn.MaxPool1d(2),
    nn.Dropout(0.5),
    nn.Conv1d(32,64,kernel_size=(3),padding=1,stride=2),
    nn.ReLU(),
    nn.BatchNorm1d(64),
    nn.Conv1d(64,64,kernel_size=(3),padding=1,stride=2),
    nn.ReLU(),
    nn.BatchNorm1d(64),
    nn.MaxPool1d(2),
    nn.Dropout(0.5),
    nn.Flatten(),
    nn.Linear(1024,512),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(512,85),
    nn.Sigmoid()
)

In [9]:
class MyNet(nn.Module):
    def __init__(self,backbone,fully_connected):
        super().__init__()
        self.backbone = backbone
        self.fully_connected = fully_connected

    def forward(self,x):
        vector = torch.zeros((x.size(0),limit,1000))
        i = 0
        for trailer in x:
            output = self.backbone(trailer) 
            vector[i] = output
            i +=1
        result = self.fully_connected(vector.to(device))
        return result

In [10]:
model = MyNet(net,fully_connected)

for params in model.parameters():
  params.requires_grad = False

model = model.to(device)

print('Numero parametri model: ',count_parameters(model))

model

Numero parametri model:  0


MyNet(
  (backbone): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (16):

In [11]:
load_model(model)

Caricamento modello avvenuto con successo!


### Testing del Modello Ottenuto

In [12]:
tk = [10, 7, 5]

for i in range(len(tk)):
    test(model, topk=tk[i])
    print()

Start testing the model ( topk =  10 )
Test END: -- m_precision =  0.17  -- m_recall =  0.6  -- m_f1 =  0.27

Start testing the model ( topk =  7 )
Test END: -- m_precision =  0.21  -- m_recall =  0.52  -- m_f1 =  0.3

Start testing the model ( topk =  5 )
Test END: -- m_precision =  0.25  -- m_recall =  0.43  -- m_f1 =  0.31



In [13]:
topk=10
predictions = []
y_true = []

model.eval()

with torch.no_grad():
    for inputs, labels in testDataLoader:
        inputs = inputs.to(device)

        y_pred = model(inputs)

        _, idx = y_pred.topk(topk, dim=1)

        y_pred = torch.zeros_like(y_pred)
        y_pred.scatter_(1, idx, 1)
        predictions.append(y_pred.cpu())

        y_true.append(labels.cpu())


y_true, predictions = torch.cat(y_true, axis=0), torch.cat(predictions, axis=0)
report = classification_report(y_true, predictions, 
                               target_names=list(sorted(label_idx.keys())),zero_division=0)
print(report)

                      precision    recall  f1-score   support

                LGBT       0.08      0.72      0.14        61
              action       0.22      0.94      0.36       191
     action_comedies       0.00      0.00      0.00         6
           adventure       0.13      0.76      0.22        75
   alcohol_addiction       0.00      0.00      0.00         6
               alien       0.00      0.00      0.00        12
           animation       0.23      0.86      0.36        21
            aviation       0.00      0.00      0.00        20
        bank_robbery       0.00      0.00      0.00        19
              biopic       0.09      0.32      0.14        59
             cartoon       0.10      0.50      0.17         8
               chase       0.29      0.12      0.17        16
            children       0.13      0.25      0.17        12
              comedy       0.19      0.99      0.32       211
        comedy_drama       0.07      0.77      0.13        65
       

Risultati ottenuti: micro average f1 score 0.27 con 41 classi con f1 score maggiore di 0.